In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import dizx

In [2]:
g = dizx.Graph(3)

In [3]:
v = g.add_vertex(1)

In [4]:
w = g.add_vertex(2)

In [5]:
eo = dizx.Edge(3)

In [6]:
e = g.add_edge(g.edge(v,w),eo)

In [8]:
g

Graph(2 vertices, 1 edges, dimension 3)